In [1]:
from mysql.sqlConnect import sqlConnection,sqlDisconnection,sqlSelect, sqlInsert, columnName

In [16]:
cur=sqlConnection()
data=sqlSelect("SELECT * FROM player WHERE name<>%s limit 3","x")
column=columnName("player")
res={"data":data,"column":column}

for val_i in data:
    for val_j in val_i:
        print(val_j)
        

#sqlDisconnection()

success: effect row: 3
success: effect row: 5
106863163
Lu Yao 路垚
SoMnus丶M
CN
2
101695162
Xu Linsen 徐林森
FY
CN
4
131043881
Xiao Zuoyi
RAI
CN
4


SyntaxError: invalid syntax (<ipython-input-1-aed1063c71a5>, line 1)

In [7]:
from mysql.sqlConnect import sqlConnection,sqlDisconnection,sqlInsert
from mysql.dataHandler import api_transfer_sql,sqlColumn
##写入MySQL
cur=sqlConnection()

    #插入数据
sql="""
'insert into match(barracks_status_dire,barracks_status_radiant,cluster,dire_score,duration,engine,first_blood_time,flags,game_mode,human_players,league_id,lobby_type,match_id,match_seq_num,negative_votes,positive_votes,pre_game_duration,radiant_score,radiant_win,start_time,tower_status_dire,tower_status_radiant,season,skill) 
values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
"""

param="""
"""


row=sqlInsert(sql,param)
print("插入数据")
print(row)
    
    #关闭数据库
sqlDisconnection()

success: effect row: 3
插入数据
3
close success


In [21]:
import time

int(time.mktime(time.strptime("2020-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")))

1577808000

In [12]:
"asd"[:-1]

'as'